# Convert from YOLO to Pascal VOC

This script processes YOLO annotations and converts them into the Pascal VOC format. It reads bounding boxes from YOLO-style .txt files, converts them to Pascal VOC format, and saves them in .xml files alongside the images.

## Pascal VOC format for Object Detection with Landing Lens
If you have labeled images you want to upload to Object Detection projects, you can upload them in the Pascal VOC (Visual Object Classes) format. This format involves uploading the original (unlabeled) image and a corresponding XML file. The XML file contains the label (annotation) details of its paired image. The XML file essentially tells LandingLens where each label is on its associated image and what the name of the class is.

The image and XML file must have the same file name (with different extensions). For example:

vehicle_123.png
vehicle_123.xml

Additional details at:
https://support.landing.ai/docs/upload-labeled-images-od?highlight=pascal%20voc


In [ ]:
import argparse
import os
import sys
import shutil
import cv2
from lxml import etree, objectify
from tqdm import tqdm

## Define functions

### save_anno_to_xml()

save_anno_to_xml(filename, size, objs, save_path)

**Parameters:**
* filename: The image filename.
* size: The dimensions of the image (height, width, depth).
* objs: A list of objects, each containing a category and bounding box coordinates.
* save_path: The directory where the .xml files will be saved.

Converts annotations into Pascal VOC .xml format and saves them. It uses the objectify.ElementMaker to generate XML elements and builds an XML tree with annotation details like the image name, size, and bounding box information.
The resulting XML is saved with the same name as the image, but with a .xml extension.

In [ ]:
def save_anno_to_xml(filename, size, objs, save_path):
    E = objectify.ElementMaker(annotate=False)
    anno_tree = E.annotation(
        E.folder("DATA"),
        E.filename(filename),
        E.source(
            E.database("The VOC Database"),
            E.annotation("PASCAL VOC"),
            E.image("flickr")
        ),
        E.size(
            E.width(size[1]),
            E.height(size[0]),
            E.depth(size[2])
        ),
        E.segmented(0)
    )
    for obj in objs:
        E2 = objectify.ElementMaker(annotate=False)
        anno_tree2 = E2.object(
            E.name(obj[0]),
            E.pose("Unspecified"),
            E.truncated(0),
            E.difficult(0),
            E.bndbox(
                E.xmin(obj[1][0]),
                E.ymin(obj[1][1]),
                E.xmax(obj[1][2]),
                E.ymax(obj[1][3])
            )
        )
        anno_tree.append(anno_tree2)
    anno_path = os.path.join(save_path, filename[:-3] + "xml")
    etree.ElementTree(anno_tree).write(anno_path, pretty_print=True)

### xywhn2xyxy()

xywhn2xyxy(bbox, size)

**Parameters:**
* bbox: A YOLO-format bounding box as (center_x, center_y, width, height), normalized to the image dimensions.
* size: The actual dimensions of the image.

Converts YOLO-format bounding boxes (center coordinates, width, height as normalized values) to Pascal VOC-style bounding boxes (xmin, ymin, xmax, ymax in pixel values). The function calculates the pixel positions of the bounding box edges and returns them as integers.

In [ ]:
def xywhn2xyxy(bbox, size):
    bbox = list(map(float, bbox))
    size = list(map(float, size))
    xmin = (bbox[0] - bbox[2] / 2.) * size[1]
    ymin = (bbox[1] - bbox[3] / 2.) * size[0]
    xmax = (bbox[0] + bbox[2] / 2.) * size[1]
    ymax = (bbox[1] + bbox[3] / 2.) * size[0]
    box = [xmin, ymin, xmax, ymax]
    return list(map(int, box))

### parseXmlFilse()

parseXmlFilse(image_path, anno_path, save_path)

**Parameters**
* image_path: Path to the folder containing images.
* anno_path: Path to the folder containing YOLO-format .txt annotation files.
* save_path: Path where the resulting Pascal VOC .xml files will be saved.

The main function that processes the YOLO .txt annotation files, converts them to Pascal VOC .xml files, and saves them. Key steps are 1) reading the classes from classes.txt to build a category set, 2) Mapping each image file and its annotations, 3)
iterating over each annotation file and reading the bounding box data, converting it to VOC format using xywhn2xyxy(), and 4)
calling save_anno_to_xml() to save each annotation file as an XML.

In [ ]:
def parseXmlFilse(image_path, anno_path, save_path):
    global images_nums, category_nums, bbox_nums
    assert os.path.exists(image_path), "ERROR {} dose not exists".format(image_path)
    assert os.path.exists(anno_path), "ERROR {} dose not exists".format(anno_path)
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path)

    category_set = []
    with open(anno_path + '/classes.txt', 'r') as f:
        for i in f.readlines():
            category_set.append(i.strip())
    category_nums = len(category_set)
    category_id = dict((k, v) for k, v in enumerate(category_set))

    images = [os.path.join(image_path, i) for i in os.listdir(image_path)]
    files = [os.path.join(anno_path, i) for i in os.listdir(anno_path)]
    images_index = dict((v.split(os.sep)[-1][:-4], k) for k, v in enumerate(images))
    images_nums = len(images)

    for file in tqdm(files):
        if os.path.splitext(file)[-1] != '.txt' or 'classes' in file.split(os.sep)[-1]:
            continue
        if file.split(os.sep)[-1][:-4] in images_index:
            index = images_index[file.split(os.sep)[-1][:-4]]
            img = cv2.imread(images[index])
            shape = img.shape
            filename = images[index].split(os.sep)[-1]
        else:
            continue
        objects = []
        with open(file, 'r') as fid:
            for i in fid.readlines():
                i = i.strip().split()
                category = int(i[0])
                category_name = category_id[category]
                bbox = xywhn2xyxy((i[1], i[2], i[3], i[4]), shape)
                obj = [category_name, bbox]
                objects.append(obj)
        bbox_nums += len(objects)
        save_anno_to_xml(filename, shape, objects, save_path)

## Main Block

if __name__ == '__main__':

* Parses command-line arguments using argparse for paths:
--anno-path: Path to the YOLO .txt annotations.
--save-path: Destination folder for the VOC .xml files.
--image-path: Path to the images.
* If no command-line arguments are provided, default paths are used.
* After parsing the arguments, the script calls parseXmlFilse with the provided arguments or default paths to begin the conversion process.
* Finally, it prints the total number of images, categories, and bounding boxes processed.


In [ ]:
images_nums = 0
category_nums = 0
bbox_nums = 0

if __name__ == '__main__':
    """
    Script Description:
        This script is used to convert annotation file .txt in yolo format to annotation file .xml in voc format
    Parameter description:
        anno_path:txt storage path of the annotation file.
        save_path:the folder where the json file will be output.
        image_path:path of the image.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('-ap', '--anno-path', type=str, default='./data/labels/yolo', help='yolo txt path')
    parser.add_argument('-s', '--save-path', type=str, default='./data/convert/voc', help='xml save path')
    parser.add_argument('--image-path', default='./data/images')

    opt = parser.parse_args()
    if len(sys.argv) > 1:
        print(opt)
        parseXmlFilse(**vars(opt))
        print("image nums: {}".format(images_nums))
        print("category nums: {}".format(category_nums))
        print("bbox nums: {}".format(bbox_nums))
    else:
        anno_path = './data/labels/yolo'
        save_path = './data/convert/voc1'
        image_path = './data/images'
        parseXmlFilse(image_path, anno_path, save_path)
        print("image nums: {}".format(images_nums))
        print("category nums: {}".format(category_nums))
        print("bbox nums: {}".format(bbox_nums))


